# Scratchpad for paper revisions

In [1]:
%load_ext autoreload
%autoreload 2
import pickle
import os, sys
root_path = os.path.realpath('../')
sys.path.append(root_path)

import torch
from pathlib import Path
import numpy as np
import random


from utils.data import make_blobs_dataset, make_trees_dataset
from utils.nnet import get_device

from hebbcl.logger import LoggerFactory
from hebbcl.model import Nnet, ScaledNet2Hidden
from hebbcl.trainer import Optimiser, train_on_blobs, train_on_trees
from hebbcl.parameters import parser
from hebbcl.tuner import HPOTuner

## Hyperparameter optimisation
hpo on network trained with fewer episodes

### HPO: blocked trials with oja_ctx

In [ ]:
# HPO on blocked trials with oja_ctx
args = parser.parse_args(args=[])
args.n_episodes = 8
args.hpo_fixedseed = True
args.hpo_scheduler = "bohb"
args.hpo_searcher = "bohb"
# dict(sorted(vars(args).items(),key=lambda k: k[0]))
args.ctx_avg = False
# init tuner
tuner = HPOTuner(args, time_budget=60*15, metric="loss")

tuner.tune(n_samples=500)

df = tuner.results
df = df[["mean_loss", "mean_acc", "config.lrate_sgd","config.lrate_hebb", "config.ctx_scaling","config.seed","done"]]
df = df[df["done"]==True]
df = df.drop(columns=["done"])
df = df.dropna()
df = df.sort_values("mean_loss",ascending=True)

df.reset_index()
print(df.head(15))

print(tuner.best_cfg)

with open("../results/raytune_oja_ctx_blocked_8episodes.pkl", "wb") as f:
    pickle.dump(df, f)

In [ ]:
with open("../results/raytune_oja_ctx_blocked_8episodes.pkl", "rb") as f:
    df = pickle.load(f)

df.iloc[0]

In [ ]:
# verify results 
with open("../results/raytune_oja_ctx_blocked_8episodes.pkl", "rb") as f:
    df = pickle.load(f)
# obtain params
args = parser.parse_args(args=[])

# set checkpoint directory
save_dir = (
        Path("checkpoints") / "test_allhebb"
    ) 

# get device (gpu/cpu)
args.device = get_device(args.cuda)[0]

# override defaults 
args.n_episodes = 8
args.lrate_hebb = df.iloc[0]["config.lrate_hebb"]
args.lrate_sgd = df.iloc[0]["config.lrate_sgd"]
args.ctx_scaling = df.iloc[0]["config.ctx_scaling"]
args.ctx_avg = False
np.random.seed(int(df.iloc[0]["config.seed"]))
random.seed(int(df.iloc[0]["config.seed"]))
torch.manual_seed(int(df.iloc[0]["config.seed"]))


# create dataset 
dataset = make_blobs_dataset(args)

# instantiate logger, model and optimiser:
logger = LoggerFactory.create(args, save_dir)
model = Nnet(args)
optimiser = Optimiser(args)

# send model to device (GPU?)
model = model.to(args.device)


# train model
train_on_blobs(args, model, optimiser, dataset, logger)

print(f"config: lrate_sgd: {args.lrate_sgd:.4f}, lrate_hebb: {args.lrate_hebb:.4f}, context offset: {args.ctx_scaling}")
print(f"terminal accuracy: {logger.results['acc_total'][-1]:.2f}, loss: {logger.results['losses_total'][-1]:.2f}")

### HPO: Interleaved trials

In [ ]:
# HPO on blocked trials with oja_ctx
args = parser.parse_args(args=[])
args.n_episodes = 8
args.hpo_fixedseed = True
args.hpo_scheduler = "bohb"
args.hpo_searcher = "bohb"
args.training_schedule = "interleaved"
# dict(sorted(vars(args).items(),key=lambda k: k[0]))
args.ctx_avg = False
# init tuner
tuner = HPOTuner(args, time_budget=60*15, metric="loss")

tuner.tune(n_samples=500)

df = tuner.results
df = df[["mean_loss", "mean_acc", "config.lrate_sgd","config.lrate_hebb", "config.ctx_scaling","config.seed","done"]]
df = df[df["done"]==True]
df = df.drop(columns=["done"])
df = df.dropna()
df = df.sort_values("mean_loss",ascending=True)

df.reset_index()
print(df.head(15))

print(tuner.best_cfg)

with open("../results/raytune_oja_ctx_interleaved_8episodes.pkl", "wb") as f:
    pickle.dump(df, f)

In [ ]:
# verify results 

# obtain params
args = parser.parse_args(args=[])

# set checkpoint directory
save_dir = (
        Path("checkpoints") / "test_allhebb"
    ) 

# get device (gpu/cpu)
args.device = get_device(args.cuda)[0]

# override defaults 
args.n_episodes = 8
args.lrate_hebb = df.iloc[0]["config.lrate_hebb"]
args.lrate_sgd = df.iloc[0]["config.lrate_sgd"]
args.ctx_scaling = df.iloc[0]["config.ctx_scaling"]
args.ctx_avg = False
args.training_schedule = "interleaved"
np.random.seed(int(df.iloc[0]["config.seed"]))
random.seed(int(df.iloc[0]["config.seed"]))
torch.manual_seed(int(df.iloc[0]["config.seed"]))



# create dataset 
dataset = make_blobs_dataset(args)

# instantiate logger, model and optimiser:
logger = LoggerFactory.create(args, save_dir)
model = Nnet(args)
optimiser = Optimiser(args)

# send model to device (GPU?)
model = model.to(args.device)


# train model
train_on_blobs(args, model, optimiser, dataset, logger)

print(f"config: lrate_sgd: {args.lrate_sgd:.4f}, lrate_hebb: {args.lrate_hebb:.4f}, context offset: {args.ctx_scaling}")
print(f"terminal accuracy: {logger.results['acc_total'][-1]:.2f}, loss: {logger.results['losses_total'][-1]:.2f}")

### Trees

In [39]:
with open("../results/raytune_trees_interleaved_vanilla_1ctx.pkl","rb") as f:
    df = pickle.load(f)["df"]
df.sort_values("mean_loss").head(15)


,mean_loss,mean_acc,done,config.lrate_sgd,config.ctx_scaling,config.seed
trial_id,,,,,,
c1e11c40,-5179.713379,0.84,True,0.000793,4,6112
c9eb260f,-435.491669,0.50,False,0.002728,1,9885
c9ed0ad4,-225.550491,0.50,False,0.002233,2,8350
c9dc93c9,-137.268188,0.50,False,0.001514,5,5642
c1bf8182,-74.009750,0.50,False,0.001368,3,7425
c9ef14d5,-61.272003,0.50,True,0.002444,5,305
b2840999,-47.112179,0.50,False,0.000922,2,4403
be5c0ef5,-31.493992,0.50,False,0.001812,3,6468
c7d5978e,-27.358358,0.50,False,0.000568,1,7970


In [13]:
best_config = dict(df.iloc[2,:])
best_config

{'mean_loss': -5288.3916015625,
 'mean_acc': 0.8424999713897705,
 'done': True,
 'config.lrate_sgd': 0.001405253754308637,
 'config.lrate_hebb': 0.0001261706686407157,
 'config.ctx_scaling': 5,
 'config.seed': 2789}

In [14]:

# obtain params
args = parser.parse_args(args=[])

# set checkpoint directory
save_dir = (
        Path("checkpoints") / "test_allhebb"
    ) 

# get device (gpu/cpu)
args.device = get_device(args.cuda)[0]

# override defaults 
args.n_episodes = 100
args.n_layers = 2
args.n_hidden = 100
args.n_features = 974
args.ctx_avg = False
args.lrate_hebb = best_config["config.lrate_hebb"]
args.lrate_sgd = best_config["config.lrate_sgd"]
args.ctx_scaling = best_config["config.ctx_scaling"]

args.ctx_twice = False
args.training_schedule = "blocked"
args.perform_hebb = True
args.centering = True
args.gating = "oja"

np.random.seed(best_config["config.seed"])
random.seed(best_config["config.seed"])
torch.manual_seed(best_config["config.seed"])



# create dataset 
dataset = make_trees_dataset(args)

# instantiate logger, model and optimiser:
logger = LoggerFactory.create(args, save_dir)
model = ScaledNet2Hidden(args)
optimiser = Optimiser(args)

# send model to device (GPU?)
model = model.to(args.device)


# train model
train_on_trees(args, model, optimiser, dataset, logger)

print(f"config: lrate_sgd: {args.lrate_sgd:.4f}, lrate_hebb: {args.lrate_hebb:.4f}, context offset: {args.ctx_scaling}")
print(f"terminal accuracy: {logger.results['acc_total'][-1]:.2f}, loss: {logger.results['losses_total'][-1]:.2f}")

step 0, loss: task a 0.5561, task b 0.0699 | acc: task a 0.5000, task b 0.5000
...1st hidden: n_a: 7 n_b: 1
... 2nd hidden: n_a: 6 n_b: 2
step 50, loss: task a -0.1111, task b 0.0529 | acc: task a 0.5000, task b 0.5000
...1st hidden: n_a: 6 n_b: 2
... 2nd hidden: n_a: 4 n_b: 3
step 100, loss: task a -1.2556, task b 0.3045 | acc: task a 0.5000, task b 0.5000
...1st hidden: n_a: 3 n_b: 3
... 2nd hidden: n_a: 7 n_b: 2
step 150, loss: task a -4.6015, task b 2.7430 | acc: task a 0.5000, task b 0.5000
...1st hidden: n_a: 3 n_b: 2
... 2nd hidden: n_a: 4 n_b: 2
step 200, loss: task a -8.4972, task b 6.0427 | acc: task a 0.5000, task b 0.5000
...1st hidden: n_a: 3 n_b: 1
... 2nd hidden: n_a: 5 n_b: 0
step 250, loss: task a -32.5606, task b 21.6849 | acc: task a 0.5000, task b 0.5000
...1st hidden: n_a: 4 n_b: 0
... 2nd hidden: n_a: 1 n_b: 1
step 300, loss: task a -58.2328, task b 49.9133 | acc: task a 0.5000, task b 0.5000
...1st hidden: n_a: 4 n_b: 0
... 2nd hidden: n_a: 1 n_b: 0
step 350, los